Welcome to the MLP classification notebook. In this notebook we will be exploring two new items. 

* The first will be learning how to use the MultiLayerPerceptron Algorithm. In the case that we start introducing hidden layers one would call this "Deep Learning" and "Deep Neural Network" which is an extremely powerful modeling technique. 

* The second concept which is extremely relevant for DNN is the selection of and tuning of hyperparameters. This selection can be computationally expensive and time consuming. For this we introduce sklearn's gridsearchcv to help us search/explore the parameter space without having to code this manually. 

* The notebook will begin by simply creating MLP models with various numbers of layers and a variety of neurons in each layer with the models becoming more computationally expensive and complex. Pay attention to the metrics to see if this complexity has actually improved our predictions. DNNs are notorious for overfitting data because they can become arbitrarily complex. 

* After you have grasped the primary concept of the DNN we will introduce other hyperparameters that are often tuned to improve the model, and finally we will introduce gridsearchcv to bring this code complexity back down to a reasonable effort. 

## Setup

In [ ]:
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix,\
 recall_score, precision_score, f1_score, accuracy_score, make_scorer,\
  precision_recall_fscore_support

from sklearn.model_selection import train_test_split, cross_validate

from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data

In [ ]:
titanic_cleaned = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/6482_to_4482/titanic_cleaned.csv').drop('Cabin', axis=1) # drop cabin

In [ ]:
titanic_cleaned.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [ ]:
titanic_cleaned['Pclass'] = titanic_cleaned.Pclass.astype(str)

In [ ]:
titanic_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  714 non-null    int64  
 1   Pclass    714 non-null    object 
 2   Sex       714 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     714 non-null    int64  
 5   Parch     714 non-null    int64  
 6   Fare      714 non-null    float64
 7   Embarked  714 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 44.8+ KB


In [ ]:
y = titanic_cleaned.pop('Survived')

In [ ]:
X = pd.get_dummies(titanic_cleaned)
print(X.shape, y.shape)

(714, 13) (714,)


In [ ]:
X.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_missing
0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1,0
1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0,0
2,26.0,0,0,7.9250,0,0,1,1,0,0,0,1,0
3,35.0,1,0,53.1000,1,0,0,1,0,0,0,1,0
4,35.0,0,0,8.0500,0,0,1,0,1,0,0,1,0


## MLP

# Changing the number of hidden layers on the model

### Model 1 (no hidden layers)

In [ ]:
model_1 = MLPClassifier(random_state=2021,hidden_layer_sizes=()).fit(X,y)


In [ ]:
model_1

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=2021, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
model_1.n_layers_

2

In [ ]:
model_1.hidden_layer_sizes

()

In [ ]:
model_1.classes_

array([0, 1])

In [ ]:
len(model_1.coefs_)

1

In [ ]:
model_1.coefs_[0].shape

(13, 1)

In [ ]:
model_1.coefs_[0]

array([[-0.02392046],
       [ 0.12518862],
       [-0.13776903],
       [ 0.00281199],
       [ 0.86980426],
       [ 0.08472776],
       [-0.74070596],
       [ 0.87249304],
       [-0.33628334],
       [ 0.3924393 ],
       [-0.65300024],
       [-0.37585229],
       [ 0.82755228]])

In [ ]:
model_1_cv_results = pd.DataFrame(cross_validate(model_1,
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

model_1_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.153169,0.004019,0.651261,0.756303,0.427083,0.561856,0.594203,0.778571,0.496970,0.652695
1,0.159221,0.004163,0.726891,0.735294,0.505155,0.533679,0.742424,0.741007,0.601227,0.620482
2,0.141638,0.003806,0.815126,0.731092,0.639175,0.518135,0.873239,0.740741,0.738095,0.609756


### Model 2 (1 hidden layer with 50 neurons)

In [ ]:
# why do we fit here? without fitting we have no layers
model_2 = MLPClassifier(random_state=2021,hidden_layer_sizes=(50,)).fit(X,y)
print("hidden layers sizes",model_2.hidden_layer_sizes)
print("n_layers_",model_2.n_layers_)

hidden layers sizes (50,)
n_layers_ 3


In [ ]:
model_2_cv_results = pd.DataFrame(cross_validate(model_2,
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

model_2_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.510523,0.005917,0.773109,0.842437,0.677083,0.762887,0.738636,0.836158,0.706522,0.797844
1,0.463826,0.006608,0.773109,0.823529,0.701031,0.746114,0.731183,0.804469,0.715789,0.774194
2,0.475706,0.005845,0.794118,0.804622,0.701031,0.740933,0.772727,0.768817,0.735135,0.754617


### Model 3 (2 hidden layers, the first has 15 neurons and the second has 10 neurons)


In [ ]:
# why do we fit here? without fitting we have no layers
model_3 = MLPClassifier(random_state=2021,hidden_layer_sizes=(15,10)).fit(X,y)
print("hidden layers sizes",model_3.hidden_layer_sizes)
print("n_layers_",model_3.n_layers_)

hidden layers sizes (15, 10)
n_layers_ 4


In [ ]:
model_3_cv_results = pd.DataFrame(cross_validate(model_3,
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

model_3_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.357740,0.004523,0.747899,0.838235,0.635417,0.762887,0.709302,0.826816,0.670330,0.793566
1,0.325292,0.003986,0.789916,0.819328,0.690722,0.725389,0.770115,0.809249,0.728261,0.765027
2,0.342273,0.004065,0.789916,0.815126,0.680412,0.720207,0.776471,0.803468,0.725275,0.759563


### Model 3 (3 hidden layers, the first has 50 neurons , the second has 25 neurons, the third has 10 neurons)

In [ ]:
# why do we fit here? without fitting we have no layers
model_4 = MLPClassifier(random_state=2021,hidden_layer_sizes=(50,25,10)).fit(X,y)
print("hidden layers sizes",model_4.hidden_layer_sizes)
print("n_layers_",model_4.n_layers_)

hidden layers sizes (50, 25, 10)
n_layers_ 5


In [ ]:
model_4_cv_results = pd.DataFrame(cross_validate(model_4,
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

model_4_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.518258,0.012648,0.731092,0.840336,0.708333,0.840206,0.653846,0.783654,0.680000,0.810945
1,0.594522,0.006298,0.802521,0.852941,0.701031,0.746114,0.790698,0.872727,0.743169,0.804469
2,0.563333,0.006289,0.815126,0.836134,0.721649,0.730570,0.804598,0.844311,0.760870,0.783333


### Model 5 (4 hidden layers, the first has 100 neurons , the second has 50 neurons, the third has 25 neurons and the fourth has 10 neurons)

In [ ]:
# why do we fit here? without fitting we have no layers
model_5 = MLPClassifier(random_state=2021,hidden_layer_sizes=(100,50,25,10)).fit(X,y)
print("hidden layers sizes",model_5.hidden_layer_sizes)
print("n_layers_",model_5.n_layers_)

hidden layers sizes (100, 50, 25, 10)
n_layers_ 6


In [ ]:
model_5_cv_results = pd.DataFrame(cross_validate(model_5,
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

model_5_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.661943,0.007113,0.743697,0.842437,0.614583,0.737113,0.710843,0.856287,0.659218,0.792244
1,0.679306,0.006596,0.789916,0.836134,0.731959,0.803109,0.747368,0.794872,0.739583,0.798969
2,0.626938,0.006618,0.827731,0.821429,0.711340,0.704663,0.841463,0.829268,0.770950,0.761905


### Model 6 (5 hidden layers, the first has 100 neurons , the second has 50 neurons, the third has 25 neurons , the fourth has 25 neurons , the fifth has 10 neurons)

In [ ]:
model_6_cv_results = pd.DataFrame(cross_validate(MLPClassifier(random_state=2021,hidden_layer_sizes=(100,50,25,25,10)),
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

model_6_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.897319,0.008431,0.735294,0.848739,0.677083,0.840206,0.670103,0.799020,0.673575,0.819095
1,1.305975,0.008022,0.802521,0.821429,0.752577,0.803109,0.760417,0.767327,0.756477,0.784810
2,1.064815,0.006546,0.798319,0.813025,0.721649,0.746114,0.769231,0.782609,0.744681,0.763926


### Model 7 (3 hidden layers each with 500 neurons)

notice how much longer this model took to train compared to the others!

In [ ]:
model_7_cv_results = pd.DataFrame(cross_validate(MLPClassifier(random_state=2021,hidden_layer_sizes=(500,500,500)),
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

model_7_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,12.012073,0.017334,0.752101,0.863445,0.687500,0.845361,0.694737,0.824121,0.691099,0.834606
1,5.379547,0.016595,0.806723,0.800420,0.721649,0.715026,0.786517,0.775281,0.752688,0.743935
2,9.155703,0.016759,0.777311,0.806723,0.577320,0.637306,0.823529,0.848276,0.678788,0.727811


In [ ]:
model_1_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.153169,0.004019,0.651261,0.756303,0.427083,0.561856,0.594203,0.778571,0.496970,0.652695
1,0.159221,0.004163,0.726891,0.735294,0.505155,0.533679,0.742424,0.741007,0.601227,0.620482
2,0.141638,0.003806,0.815126,0.731092,0.639175,0.518135,0.873239,0.740741,0.738095,0.609756


In [ ]:
model_2_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.510523,0.005917,0.773109,0.842437,0.677083,0.762887,0.738636,0.836158,0.706522,0.797844
1,0.463826,0.006608,0.773109,0.823529,0.701031,0.746114,0.731183,0.804469,0.715789,0.774194
2,0.475706,0.005845,0.794118,0.804622,0.701031,0.740933,0.772727,0.768817,0.735135,0.754617


In [ ]:
model_3_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.357740,0.004523,0.747899,0.838235,0.635417,0.762887,0.709302,0.826816,0.670330,0.793566
1,0.325292,0.003986,0.789916,0.819328,0.690722,0.725389,0.770115,0.809249,0.728261,0.765027
2,0.342273,0.004065,0.789916,0.815126,0.680412,0.720207,0.776471,0.803468,0.725275,0.759563


In [ ]:
model_4_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.518258,0.012648,0.731092,0.840336,0.708333,0.840206,0.653846,0.783654,0.680000,0.810945
1,0.594522,0.006298,0.802521,0.852941,0.701031,0.746114,0.790698,0.872727,0.743169,0.804469
2,0.563333,0.006289,0.815126,0.836134,0.721649,0.730570,0.804598,0.844311,0.760870,0.783333


In [ ]:
model_5_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.661943,0.007113,0.743697,0.842437,0.614583,0.737113,0.710843,0.856287,0.659218,0.792244
1,0.679306,0.006596,0.789916,0.836134,0.731959,0.803109,0.747368,0.794872,0.739583,0.798969
2,0.626938,0.006618,0.827731,0.821429,0.711340,0.704663,0.841463,0.829268,0.770950,0.761905


In [ ]:
model_6_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,0.897319,0.008431,0.735294,0.848739,0.677083,0.840206,0.670103,0.799020,0.673575,0.819095
1,1.305975,0.008022,0.802521,0.821429,0.752577,0.803109,0.760417,0.767327,0.756477,0.784810
2,1.064815,0.006546,0.798319,0.813025,0.721649,0.746114,0.769231,0.782609,0.744681,0.763926


In [ ]:
model_7_cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,12.012073,0.017334,0.752101,0.863445,0.687500,0.845361,0.694737,0.824121,0.691099,0.834606
1,5.379547,0.016595,0.806723,0.800420,0.721649,0.715026,0.786517,0.775281,0.752688,0.743935
2,9.155703,0.016759,0.777311,0.806723,0.577320,0.637306,0.823529,0.848276,0.678788,0.727811


Analyzing the results as shown above notice that the performance of the models is not always increasing even though the complexity is increasing. The increased complexity does seem to increase the fit time though which is expected. A more complex model will take longer to train. So there is a trade off between quality of the model and complexity of the model. 

In addition, notice how much code we had to write just to get these results? And if you were starting to feel like the code is redundant for each section you are correct! Notice with a simple function how we can do exactly the same thing much easier as shown below. 

In [ ]:
model_1_hidden_layers = ()
model_2_hidden_layers = (50,)
model_3_hidden_layers = (15,10)
model_4_hidden_layers = (50,25,10)
model_5_hidden_layers = (100,50,25,10)
model_6_hidden_layers = (100,50,25,25,10)
model_7_hidden_layers = (500,500,500)
model_8_hidden_layers = (500,500,500,500)

In [ ]:
# create a function to train/evaluate the model using cross validation and return the
# results as a dataframe
def cv_mlp_models(hidden_layer_param):
  return_df = pd.DataFrame(cross_validate(MLPClassifier(random_state=2021,hidden_layer_sizes=hidden_layer_param),
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))
  return_df['hidden'] = str(hidden_layer_param)
  return return_df

In [ ]:
# now simply join all the result dataframes together
def join_dfs(df,concat_df):
  concat_df = concat_df.append(df)
  return concat_df

In [ ]:
# make a list of the hidden layer tuples so we can interate over it with a for loop
list_of_hidden_layers = [model_1_hidden_layers,model_2_hidden_layers,model_3_hidden_layers,model_4_hidden_layers,model_5_hidden_layers,model_6_hidden_layers,model_7_hidden_layers,model_8_hidden_layers]

In [ ]:
# create an empty dataframe with the correct column names, we need this so we have a dataframe that can collect all our results
concat_df = pd.DataFrame(columns=['fit_time',	'score_time'	,'test_accuracy',	'train_accuracy',	'test_recall',	'train_recall',	'test_precision',	'train_precision',	'test_f1',	'train_f1'])

# now simply iterate over our list of hidden layers, run the cv function for each hidden layer setting, and append all the results
for model_hidden_layer in list_of_hidden_layers:
  result_df = cv_mlp_models(model_hidden_layer)
  concat_df = join_dfs(result_df,concat_df)

concat_df

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1,hidden
0,0.152327,0.004194,0.651261,0.756303,0.427083,0.561856,0.594203,0.778571,0.496970,0.652695,()
1,0.139147,0.004086,0.726891,0.735294,0.505155,0.533679,0.742424,0.741007,0.601227,0.620482,()
2,0.150652,0.004081,0.815126,0.731092,0.639175,0.518135,0.873239,0.740741,0.738095,0.609756,()
0,0.498441,0.005750,0.773109,0.842437,0.677083,0.762887,0.738636,0.836158,0.706522,0.797844,"(50,)"
1,0.472097,0.006094,0.773109,0.823529,0.701031,0.746114,0.731183,0.804469,0.715789,0.774194,"(50,)"
2,0.470001,0.007235,0.794118,0.804622,0.701031,0.740933,0.772727,0.768817,0.735135,0.754617,"(50,)"
0,0.374452,0.003965,0.747899,0.838235,0.635417,0.762887,0.709302,0.826816,0.670330,0.793566,"(15, 10)"
1,0.332799,0.003979,0.789916,0.819328,0.690722,0.725389,0.770115,0.809249,0.728261,0.765027,"(15, 10)"
2,0.326950,0.004026,0.789916,0.815126,0.680412,0.720207,0.776471,0.803468,0.725275,0.759563,"(15, 10)"
0,0.501264,0.006095,0.731092,0.840336,0.708333,0.840206,0.653846,0.783654,0.680000,0.810945,"(50, 25, 10)"


# Learning Rate Hyperparameter

Before this we were simply looking at the number of hidden layers and the quantity of neurons in this hidden layers to improve our model. Another common tuning hyperparameter is learning rate. This is the rate at which the weights are adjusted and has two primary benefits. The first is that a learning weight can impact the models ability to learn more quickly or slowly and second that it can help the model stay out of local minima and hopefully find the globbal minumum when reducing loss as it is getting tuned.

Let's do a quick exploration of this and its impact on training using a simple example

In [ ]:
pd.DataFrame(cross_validate(MLPClassifier(random_state=2021,hidden_layer_sizes=(500,500,500),
                                          learning_rate_init=0.001), # note that 0.001 is the DEFAULT learning rate
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,11.922981,0.016856,0.752101,0.863445,0.687500,0.845361,0.694737,0.824121,0.691099,0.834606
1,5.362670,0.017709,0.806723,0.800420,0.721649,0.715026,0.786517,0.775281,0.752688,0.743935
2,9.281155,0.017163,0.777311,0.806723,0.577320,0.637306,0.823529,0.848276,0.678788,0.727811


now we adjust the learning rate and make it smaller to see if we can improve the model's accuracy, precision, recall etc.  

In [ ]:
pd.DataFrame(cross_validate(MLPClassifier(random_state=2021,hidden_layer_sizes=(500,500,500),
                                          learning_rate_init=0.0001), 
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,19.333929,0.017262,0.743697,0.836134,0.687500,0.819588,0.680412,0.787129,0.683938,0.803030
1,16.215836,0.017002,0.802521,0.825630,0.762887,0.813472,0.755102,0.769608,0.758974,0.790932
2,9.751922,0.016917,0.794118,0.789916,0.762887,0.761658,0.740000,0.731343,0.751269,0.746193


it appears the model is almost as good as the original, and it took almost twice as long to train! while 18 seconds may not be too long to wait imagine this model takes 6 hours, then reducing the training time to 3 hours could be a significant advantage and if the accuracy is almost the same there's almost no disadvantage to doing so

# GridSearchCV

In addition to hidden layer size and depth, learning rate, there are also momentum and the maximum number of passes that could be tuned and MANY others! This gets overwhelming quite quickly and isn't efficient for us to try to test them all! Imagine writing the code to test the following

* 5 possible hidden layer sizes
* 6 possible learning rates
* 4 possible momentums
* 6 possible total numbers of passes

That is 5 *6 * 4 * 6 = 720 possible combinations! thats way too many to try and write by hand.

### Introducing GridSearch

GridSearch can try all of those combinations for you and simplify your code

note that I have chosen to use F1 as the primary metric for the grid search to use to evaluate the hyperparameter combinations. The grid search will return the best combination that maximizes the F1 score. 

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'learning_rate_init':[0.1,0.01,0.001],
               'hidden_layer_sizes':[(50,),(50,50,50)],
              'activation':['identity', 'logistic', 'tanh', 'relu','adam']
              }
mlp = MLPClassifier(random_state=42)
clf = GridSearchCV(mlp, parameters,scoring='f1')
clf.fit(X, y)

clf.score(X, y)


0.7766323024054982

In [ ]:
# this is a complex dataframe that shows all of the various metrics you might want to look
# into if you want to understand how the grid search worked.
# a scenario might be: what if the grid search returns the "best" model but it takes 2 days to fit? (some DNNs take this long or longer to train)
# and is only slightly better than another model which takes 10 minutes to train. you might not bother
# picking the "best" model in that case

grid_search_df = pd.DataFrame(clf.cv_results_)

print(grid_search_df.shape) # we trained 30 models! is that what you would expect?

grid_search_df.sort_values('mean_test_score',ascending=False).head() #only taking the top five rows as this is a large dataframe sort by the best f1 scores found

(30, 16)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate_init,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
12,0.155906,0.042378,0.003147,0.000146,tanh,"(50,)",0.1,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.677419,0.728814,0.752137,0.724638,0.775510,0.731704,0.032685,1
19,0.232710,0.037412,0.003025,0.000114,relu,"(50,)",0.01,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.672000,0.779661,0.721311,0.666667,0.816667,0.731261,0.059025,2
8,0.621004,0.011648,0.003173,0.000055,logistic,"(50,)",0.001,"{'activation': 'logistic', 'hidden_layer_sizes...",0.678261,0.788991,0.711864,0.680000,0.789474,0.729718,0.050045,3
14,0.738172,0.011553,0.003487,0.000337,tanh,"(50,)",0.001,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.637931,0.800000,0.719298,0.666667,0.806723,0.726124,0.068284,4
10,0.944038,0.104232,0.004071,0.000122,logistic,"(50, 50, 50)",0.01,"{'activation': 'logistic', 'hidden_layer_sizes...",0.677419,0.752475,0.709091,0.659341,0.826446,0.724954,0.059790,5


after the gridsearch explores all the model combinations it selects the best model set of hyperparameters to be used. Once this model is trained it may result in an improvement over our previous efforts. Notice that I have not explored EVERY possible model we created previously and have instead opted to explore a variety of other hyperparameters to see if it's possible to achieve a better prediction than we achieved previously in the notebook. 

The point to take away from this is that
1. There are an infinite number of hyperparameter combinations
1. It's impossible to explore them all by hand
1. For each combination a model must be trained



In [ ]:
# note the hyperpameters that the gridsearch discovered achieve the best results
clf.best_estimator_

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50,), learning_rate='constant',
              learning_rate_init=0.1, max_fun=15000, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=42, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [ ]:
pd.DataFrame(cross_validate(clf, 
               X,
               y,
               cv = 3,
               return_train_score=True,
               scoring=['accuracy','recall','precision','f1']))

,fit_time,score_time,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
0,54.400396,0.006706,0.735294,0.844538,0.656250,0.829897,0.677419,0.797030,0.666667,0.813131
1,54.646148,0.013651,0.802521,0.815126,0.804124,0.787565,0.735849,0.763819,0.768473,0.775510
2,57.240458,0.007371,0.815126,0.825630,0.680412,0.699482,0.835443,0.843750,0.750000,0.764873


compare this test_f1 to the test_f1 results we had from the other models. Have we improved the model's f1 score on the test set? And, was it easier than all the code we wrote before?

In [ ]:
!cp "/content/drive/Colab Notebooks/4482_classification_MLP_titanic_cleaned.ipynb" ./
!jupyter nbconvert --to html "A6_imbico_bernardo.ipynb"

cp: cannot stat '/content/drive/Colab Notebooks/4482_classification_MLP_titanic_cleaned.ipynb': No such file or directory
[NbConvertApp] WARNING | pattern u'A6_imbico_bernardo.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--no-input
    Exclude input cells and output prompts from converted document. 
    This mode is ideal for generating code-free reports.
--stdout
    Write notebook output to stdout instead of files.
--stdi